### Projekt zaliczeniowy
#### Bootcamp Data Science

#### Temat 4 - Klasyfikacja obrazów. Celem projektu jest zastosowanie algorytmów do klasyfikacji do problemu rozpoznawania rasy psów przedstawionych na zdjęciu.

Prace rozpocząłem od podzielenia zdjęć psów na 3 grupy:
* train 
* validate
* test

Grupa treningowa była najbadziej liczna i otrzymała około 85% dostepnych zdjęć, grupa validacyjna otrzymała około 10% zdjęć, a testowa około 5%

Z racji ograniczonych zasobów sprzętwych spośród 120 ras psów wybrałem 10:
* 'NORWICH_TERRIER'
* 'GERMAN_SHEPHERD'
* 'MALTESE_DOG'
* 'COLLIE'
* 'VIZSLA'
* 'KOMONDOR'
* 'WHIPPET'
* 'ESKIMO_DOG'
* 'TIBETAN_MASTIFF'
* 'WEIMARANER'

Prace prowadziłem na komputerze stacjonarnym z systemem Lunux (Dystrybucja Ubuntu Mate 18.04) wyposażonym w:
* intel i7 2600K@4.0GHz
* 16 GB RAM DDR 3 1333 MHz XMP
* Dysk SSD
* Kartę graficzną Geforce 1070 TI z 8GB GDDR

Prace rozpocząłem od podzielenia obrazków na 3 zbiory: treningowy(1276 zdjęć), validacyjny(220 zdjęć) i testowy(203 zdjęć)

Z racji ograniczonych zasobów sprzetowych oraz tego żę chcę by wyniki były porównywalne wykorzystam obrazki w rozmiarze 75x75 (najmniejszy rozmiar obrazków akceptowalny przez gotowe modele uczenia głębokiego dostępne w Keras) - część zdjęć jest w pionie a część w poziomie więc wybrałem obrazki kwadratowe


In [1]:
import pickle
import pandas as pd
import numpy as np
import pickle
import math
import os

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten, AvgPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import MobileNet
from keras.applications.xception import Xception

from keras.optimizers import Adam, SGD
from keras.regularizers import l2

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import architectures as A

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline  

Using TensorFlow backend.


In [2]:
train_data_dir = '/home/ralph/ml/Sages/Zaliczenie/Wybrane/train/'
validation_data_dir = '/home/ralph/ml/Sages/Zaliczenie/Wybrane/validate/'
test_data_dir = '/home/ralph/ml/Sages/Zaliczenie/Wybrane/test/'

multiplier = 40 # 40 razy więcej obrazków wybierzemy z data generatora
batch_size = 32
w=75
h=75

train_datagen = ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                rotation_range=30,
                                horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(h, w),
                                                    batch_size=batch_size, 
                                                    class_mode='categorical',
                                                       shuffle=True)
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                        target_size=(h, w),
                                                        batch_size=220, # liczba wszystkich obrazków w zbiorze
                                                        class_mode='categorical',
                                                       shuffle=False)
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                        target_size=(h, w),
                                                        batch_size=203, # liczba wszystkich obrazków w zbiorze
                                                        class_mode='categorical',
                                                        shuffle=False)

num_classes = train_generator.num_classes
image_shape = train_generator.image_shape

nb_train_samples = train_generator.n
nb_validation_samples = validation_generator.n
nb_test_samples = test_generator.n

Found 1276 images belonging to 10 classes.
Found 220 images belonging to 10 classes.
Found 203 images belonging to 10 classes.


In [7]:
filename = '/home/ralph/ml/Sages/Zaliczenie/final/class_indices'
outfile = open(filename,'wb')
pickle.dump(test_generator.class_indices,outfile)
outfile.close()

In [2]:
# Ustawienie parametrów przekazywanych do modeli

callbacks = [ 
    EarlyStopping(patience=3,monitor="val_loss", restore_best_weights=True),
    ReduceLROnPlateau(patience=2) # Gdy przez 2 epoki nie będzie poprawy zmniejszy LR o 10%
]

# 0.0001, decay=0.00000001
optimizer = Adam()

regularizer = l2(0.0001)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model, eval_arch_1 = A.arch_1(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_arch_1'
outfile = open(filename,'wb')
pickle.dump(eval_arch_1,outfile)
outfile.close()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 75, 75, 8)         224       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 16)        1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 32)        4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 9, 32)          0         
___________________________

In [5]:
model, eval_arch_2 = A.arch_2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_arch_2'
outfile = open(filename,'wb')
pickle.dump(eval_arch_2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 75, 75, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 18, 18, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 9, 9, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 10368)             0         
__________

In [6]:
model, eval_arch_3 = A.arch_3(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_arch_3'
outfile = open(filename,'wb')
pickle.dump(eval_arch_3,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 75, 75, 32)        896       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 37, 37, 64)        36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 18, 18, 128)       73856     
__________

In [7]:
model, eval_arch_4 = A.arch_4(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_arch_4'
outfile = open(filename,'wb')
pickle.dump(eval_arch_4,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 75, 75, 32)        896       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 37, 37, 64)        18496     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 37, 37, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 18, 18, 128)       73856     
__________

In [8]:
model, eval_arch_5 = A.arch_5(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_arch_5'
outfile = open(filename,'wb')
pickle.dump(eval_arch_5,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 75, 75, 32)        896       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 18, 18, 128)       73856     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 9, 9, 128)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 10368)             0         
__________

In [9]:
model, eval_arch_6 = A.arch_6(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_arch_6'
outfile = open(filename,'wb')
pickle.dump(eval_arch_6,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 75, 75, 32)        896       
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 37, 37, 64)        18496     
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 37, 37, 64)        36928     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 18, 18, 128)       73856     
__________

In [10]:
model, eval_arch_7 = A.arch_7(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_arch_7'
outfile = open(filename,'wb')
pickle.dump(eval_arch_7,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 75, 75, 32)        896       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 37, 37, 64)        18496     
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 37, 37, 64)        36928     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 18, 18, 128)       73856     
__________

In [11]:
model, eval_arch_8 = A.arch_8(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_arch_8'
outfile = open(filename,'wb')
pickle.dump(eval_arch_8,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 75, 75, 32)        896       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 18, 18, 128)       73856     
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 9, 9, 128)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 10368)             0         
__________

In [12]:
model, eval_arch_9 = A.arch_9(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_arch_9'
outfile = open(filename,'wb')
pickle.dump(eval_arch_9,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 75, 75, 32)        896       
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 37, 37, 64)        18496     
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 37, 37, 64)        36928     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 18, 18, 128)       73856     
__________

In [13]:
model, eval_arch_10 = A.arch_10(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_arch_10'
outfile = open(filename,'wb')
pickle.dump(eval_arch_10,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_51 (Conv2D)           (None, 75, 75, 32)        896       
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 37, 37, 64)        18496     
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 37, 37, 64)        36928     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 18, 18, 128)       73856     
__________

In [47]:
scores=[]

for folder in os.walk('/home/ralph/ml/Sages/Zaliczenie/final/results'):
    for file in folder[2]:
        filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/' + file
        infile = open(filename,'rb')
        a=pickle.load(infile)
        a.append(file)
        scores.append(a)
        infile.close()
              
for score in scores:
    print("Architektura: {} miałą acc na zbiorze testowym równe: {:.2f}".format(score[-1],score[2]))
#     tmp=[folder[0].split("-")[-1].upper()]
#     tmp.append([folder[0]+"/"+x for x in folder[2]])
#     img_path.append(tmp)

Architektura: eval_arch_9 miałą acc na zbiorze testowym równe: 0.10
Architektura: eval_arch_6 miałą acc na zbiorze testowym równe: 0.08
Architektura: eval_arch_8 miałą acc na zbiorze testowym równe: 0.09
Architektura: eval_arch_7 miałą acc na zbiorze testowym równe: 0.09
Architektura: eval_arch_10 miałą acc na zbiorze testowym równe: 0.11
Architektura: eval_arch_2 miałą acc na zbiorze testowym równe: 0.39
Architektura: eval_arch_4 miałą acc na zbiorze testowym równe: 0.10
Architektura: eval_arch_3 miałą acc na zbiorze testowym równe: 0.10
Architektura: eval_arch_1 miałą acc na zbiorze testowym równe: 0.30
Architektura: eval_arch_5 miałą acc na zbiorze testowym równe: 0.11


Jak widać powyższe rezultaty nie zachwycają, spróbujmy innego podejścia

### Transfer learning

Keras oferuje kilka wyuczonych wcześniej modeli:

    Xception
    VGG16
    VGG19
    ResNet, ResNetV2, ResNeXt
    InceptionV3
    InceptionResNetV2
    MobileNet
    MobileNetV2
    DenseNet
    NASNet

Te modele bły przygotowywana na obrazkach od 224x224 pixeli do 299x299, ale keras pozwala trenować je obrazkami zaczynającymi się od 75x75. Z racji ograniczonej mocy obliczeniowej właśnie takie obrazki zastosuje

Podejscie - będę uczył sieć w 2 fazach. W pierwszej wczytam wagi dla warstw konwolucyjnych, dodam warstwy ukryte niewyuczone warstwy ukryte. W drugiej wazie wykorzystam model model z pierwszej fazy ale odblokuje jeden lub dwa bloki warstw konwolucyjnych aby je douczyć ale ustawie oprimaizer tak by robił bardzo małe kroki

In [3]:
callbacks = [ 
    EarlyStopping(patience=3,monitor="val_loss", restore_best_weights=True),
    ReduceLROnPlateau(patience=2) # Gdy przez 2 epoki nie będzie poprawy zmniejszy LR o 10%
]
regularizer = l2(0.0001)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
optimizer = Adam()
base_model = VGG16(weights='imagenet', include_top=False, input_shape=image_shape)

for layer in base_model.layers[1:]: 
    layer.trainable = False 
model, VGG16_arch1 = A.arch_TL_1(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, base_model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG16_arch1'
outfile = open(filename,'wb')
pickle.dump(VGG16_arch1,outfile)
outfile.close()

In [7]:
# Odblokowuje osttani blok konwolucji
for layer in model.layers[0].layers[15:]: 
    layer.trainable = True
  
optimizer = Adam(0.00001, decay=0.00000001)

# model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
# for layer in model.layers[0].layers:
#     print(layer.name, layer.trainable)
    
# model.summary()

In [8]:
model, VGG16_arch1_2 = A.TL_phase2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG16_arch1_2'
outfile = open(filename,'wb')
pickle.dump(VGG16_arch1_2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               262272    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 14,978,250
Trainable params: 7,342,986
Non-trainable params: 7,635,264
_________________________________________________________________
Epoch 1/100
1595/1595 [==============================] - 62s 39ms/step - loss: 0.3003 - acc: 0.9344 - val_loss: 2.2634 - va

In [9]:
# VGG16 druga architektura

optimizer = Adam()
regularizer = l2(0.0001)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=image_shape)

for layer in base_model.layers[1:]: 
    layer.trainable = False 
model, VGG16_arch2 = A.arch_TL_2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, base_model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG16_arch2'
outfile = open(filename,'wb')
pickle.dump(VGG16_arch2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 15,768,906
Trainable params: 1,054,218
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/100
1595/1595 [==============================] - 51s 32ms/step - loss: 0.6703 - acc: 0.8462 - val_loss: 2.1255 - v

In [10]:
# VGG16 - druga faza
# Odblokowuje ostatani blok konwolucji
for layer in model.layers[0].layers[15:]: 
    layer.trainable = True
   
optimizer = Adam(0.00001, decay=0.00000001)

model, VGG16_arch2_2 = A.TL_phase2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG16_arch2_2'
outfile = open(filename,'wb')
pickle.dump(VGG16_arch2_2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 15,768,906
Trainable params: 8,133,642
Non-trainable params: 7,635,264
_________________________________________________________________
Epoch 1/100
1595/1595 [==============================] - 63s 40ms/step - loss: 0.3302 - acc: 0.9471 - val_loss: 2.4974 - va

In [11]:
# VGG16 trzecia architektura
optimizer = Adam()

base_model = VGG16(weights='imagenet', include_top=False, input_shape=image_shape)

for layer in base_model.layers[1:]: 
    layer.trainable = False 
model, VGG16_arch3 = A.arch_TL_3(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, base_model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG16_arch3'
outfile = open(filename,'wb')
pickle.dump(VGG16_arch3,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total para

In [12]:
# VGG16 - druga faza
# Odblokowuje ostatani blok konwolucji
for layer in model.layers[0].layers[15:]: 
    layer.trainable = True
   
optimizer = Adam(0.00001, decay=0.00000001)

model, VGG16_arch3_2 = A.TL_phase2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG16_arch3_2'
outfile = open(filename,'wb')
pickle.dump(VGG16_arch3_2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total para

### VGG19

In [13]:
optimizer = Adam()
base_model = VGG19(weights='imagenet', include_top=False, input_shape=image_shape)

for layer in base_model.layers[1:]: 
    layer.trainable = False 
model, VGG19_arch1 = A.arch_TL_1(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, base_model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG19_arch1'
outfile = open(filename,'wb')
pickle.dump(VGG19_arch1,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 2, 2, 512)         20024384  
_________________________________________________________________
flatten_4 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               262272    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1290      
Total params: 20,287,946
Trainable params: 263,562
Non-trainable params: 20,024,384
_________________________________________________________________
Epoch 1/100
1595/1595 [==============================] - 54s 34ms/step - loss: 1.1895 - acc: 0.6304 - val_loss: 1.9459 - val

In [17]:
# Odblokowuje ostatani blok konwolucji
for layer in model.layers[0].layers[17:]: 
    layer.trainable = True
  
optimizer = Adam(0.00001, decay=0.00000001)

# model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# for layer in model.layers[0].layers:
#     print(layer.name, layer.trainable)
# model.summary()

In [19]:
model, VGG19_arch1_2 = A.TL_phase2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG19_arch1_2'
outfile = open(filename,'wb')
pickle.dump(VGG19_arch1_2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 2, 2, 512)         20024384  
_________________________________________________________________
flatten_4 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               262272    
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1290      
Total params: 20,287,946
Trainable params: 9,702,794
Non-trainable params: 10,585,152
_________________________________________________________________
Epoch 1/100
1595/1595 [==============================] - 75s 47ms/step - loss: 0.4229 - acc: 0.8939 - val_loss: 2.3502 - v

In [4]:
# dalej arch 2 i 3 do VGG19
# a potem jeszcze ze 2-3 modele
# Jak nie zadziła dobrze to wieksze obrazki na 1-2 modeli
# potem we flasku implementacja najlepszego modelu

In [5]:
# Architektura 2
optimizer = Adam()
base_model = VGG19(weights='imagenet', include_top=False, input_shape=image_shape)

for layer in base_model.layers[1:]: 
    layer.trainable = False 
model, VGG19_arch2 = A.arch_TL_2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, base_model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG19_arch2'
outfile = open(filename,'wb')
pickle.dump(VGG19_arch2,outfile)
outfile.close()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 2, 2, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 21,078,602
Trainable params: 1,054,218
Non-trainable params: 20,024,384
_________________________________________________________________
Instruc

In [6]:
# Odblokowuje ostatani blok konwolucji
for layer in model.layers[0].layers[17:]: 
    layer.trainable = True
  
optimizer = Adam(0.00001, decay=0.00000001)

In [7]:
model, VGG19_arch2_2 = A.TL_phase2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG19_arch2_2'
outfile = open(filename,'wb')
pickle.dump(VGG19_arch2_2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 2, 2, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 21,078,602
Trainable params: 10,493,450
Non-trainable params: 10,585,152
_________________________________________________________________
Epoch 1/100
1595/1595 [==============================] - 74s 47ms/step - loss: 0.3796 - acc: 0.9336 - val_loss: 2.5112 - 

In [8]:
# Architektura 3
optimizer = Adam()
base_model = VGG19(weights='imagenet', include_top=False, input_shape=image_shape)

for layer in base_model.layers[1:]: 
    layer.trainable = False 
model, VGG19_arch3 = A.arch_TL_3(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, base_model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG19_arch3'
outfile = open(filename,'wb')
pickle.dump(VGG19_arch3,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 2, 2, 512)         20024384  
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total para

In [9]:
# Odblokowuje ostatani blok konwolucji
for layer in model.layers[0].layers[17:]: 
    layer.trainable = True
  
optimizer = Adam(0.00001, decay=0.00000001)

In [10]:
model, VGG19_arch3_2 = A.TL_phase2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG19_arch3_2'
outfile = open(filename,'wb')
pickle.dump(VGG19_arch3_2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 2, 2, 512)         20024384  
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total para

### Xception

In [11]:
optimizer = Adam()
base_model = Xception(weights='imagenet', include_top=False, input_shape=image_shape)

for layer in base_model.layers[1:]: 
    layer.trainable = False 
model, Xception_arch1 = A.arch_TL_1(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, base_model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_Xception_arch1'
outfile = open(filename,'wb')
pickle.dump(Xception_arch1,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
flatten_3 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               2359424   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 23,222,194
Trainable params: 2,360,714
Non-trainable params: 20,861,480
_________________________________________________________________
Epoch 1/100
1595/1595 [==============================] - 55s 34ms/step - loss: 0.9574 - acc: 0.7636 - val_loss: 3.4343 - v

In [17]:
# Odblokowuje ostatani blok konwolucji
for layer in model.layers[0].layers[-6:]: 
    layer.trainable = True
  
optimizer = Adam(0.00001, decay=0.00000001)

# model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# for layer in model.layers[0].layers:
#     print(layer.name, layer.trainable)
# model.summary()

In [18]:
model, VGG19_arch1_2 = A.TL_phase2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG19_arch1_2'
outfile = open(filename,'wb')
pickle.dump(VGG19_arch1_2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
flatten_3 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               2359424   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 23,222,194
Trainable params: 7,109,514
Non-trainable params: 16,112,680
_________________________________________________________________
Epoch 1/100
1595/1595 [==============================] - 62s 39ms/step - loss: 1.0880 - acc: 0.7173 - val_loss: 3.5286 - v

In [19]:
# Architektura 2
optimizer = Adam()
base_model = Xception(weights='imagenet', include_top=False, input_shape=image_shape)

for layer in base_model.layers[1:]: 
    layer.trainable = False 
model, Xception_arch2 = A.arch_TL_2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, base_model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_Xception_arch2'
outfile = open(filename,'wb')
pickle.dump(Xception_arch2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
flatten_4 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               9437696   
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 30,304,306
Trainable params: 9,442,826
Non-trainable params: 20,861,480
_________________________________________________________________
Epoch 1/100
1595/1595 [==============================] - 63s 39ms/step - loss: 1.2759 - acc: 0.8318 - val_loss: 5.1311 - v

In [20]:
# Odblokowuje ostatani blok konwolucji
for layer in model.layers[0].layers[-6:]: 
    layer.trainable = True
  
optimizer = Adam(0.00001, decay=0.00000001)

In [21]:
model, Xception_arch2_2 = A.TL_phase2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_Xception_arch2_2'
outfile = open(filename,'wb')
pickle.dump(Xception_arch2_2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
flatten_4 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               9437696   
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 30,304,306
Trainable params: 14,191,626
Non-trainable params: 16,112,680
_________________________________________________________________
Epoch 1/100
1595/1595 [==============================] - 72s 45ms/step - loss: 1.1868 - acc: 0.7405 - val_loss: 3.3447 - 

In [22]:
# Architektura 3
optimizer = Adam()
base_model = Xception(weights='imagenet', include_top=False, input_shape=image_shape)

for layer in base_model.layers[1:]: 
    layer.trainable = False 
model, Xception_arch3 = A.arch_TL_3(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, base_model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_Xception_arch3'
outfile = open(filename,'wb')
pickle.dump(Xception_arch3,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
flatten_5 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               9437696   
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               65664     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1290      
Total para

In [23]:
# Odblokowuje ostatani blok konwolucji
for layer in model.layers[0].layers[-6:]: 
    layer.trainable = True
  
optimizer = Adam(0.00001, decay=0.00000001)

In [24]:
model, Xception_arch3_2 = A.TL_phase2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_Xception_arch3_2'
outfile = open(filename,'wb')
pickle.dump(Xception_arch3_2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
flatten_5 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               9437696   
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               65664     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1290      
Total para

### Sprawdźmy który z modeli sprawuję się najlepiej

In [27]:
scores=[]

for folder in os.walk('/home/ralph/ml/Sages/Zaliczenie/final/results'):
    for file in folder[2]:
        filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/' + file
        infile = open(filename,'rb')
        a=pickle.load(infile)
        a.append(file)
        scores.append(a)
        infile.close()
              
# print(scores)
            
def getKey(item):
    return item[2]
scores = sorted(scores, key=getKey)    
        
for score in scores:
    print("Architektura: {} miałą acc na zbiorze testowym równe: {:.2f}".format(score[-1],score[2]))

Architektura: eval_arch_6 miałą acc na zbiorze testowym równe: 0.08
Architektura: eval_arch_8 miałą acc na zbiorze testowym równe: 0.09
Architektura: eval_arch_7 miałą acc na zbiorze testowym równe: 0.09
Architektura: eval_arch_9 miałą acc na zbiorze testowym równe: 0.10
Architektura: eval_arch_4 miałą acc na zbiorze testowym równe: 0.10
Architektura: eval_arch_3 miałą acc na zbiorze testowym równe: 0.10
Architektura: eval_arch_10 miałą acc na zbiorze testowym równe: 0.11
Architektura: eval_arch_5 miałą acc na zbiorze testowym równe: 0.11
Architektura: eval_arch_1 miałą acc na zbiorze testowym równe: 0.30
Architektura: eval_arch_2 miałą acc na zbiorze testowym równe: 0.39
Architektura: eval_VGG16_arch1 miałą acc na zbiorze testowym równe: 0.41
Architektura: eval_VGG16_arch2 miałą acc na zbiorze testowym równe: 0.42
Architektura: eval_Xception_arch3 miałą acc na zbiorze testowym równe: 0.43
Architektura: eval_VGG19_arch1 miałą acc na zbiorze testowym równe: 0.43
Architektura: eval_VGG19

Wydaje się że na obrazkach w rozmiarze 75x75 nie mogę przekroczyć 55%. Uczenie sieci neuronowej zbudowanej na włąsnej aruchitekturze doprowadziło mnie jedynie do 39% skuteczności, a wykorzystanie metody transfer learningu na modelu Xception z 2 Fazowym uczeniem pozwoliło na uzyskanie 54% skuteczności. 

Spróbuję douczyć całą sieć Xception w tym celu odblokuje wszystkie warstwy, a jeśli wynik nie będzie zadowalający zacznę trenować sici na obrazkach większych rozmiarów. Z doswiadczenia już wiem że obrazki 224x224 pozwalają mi na wykorzystanie modelu VGG16 ale przy bardziej złożonych modelach mój komputer zgłasza koniec zasobów, wypróbuję więc obrazki w rozmiarze 150x150.

In [28]:
# Architektura 2
optimizer = Adam()
base_model = Xception(weights='imagenet', include_top=False, input_shape=image_shape)

for layer in base_model.layers[1:]: 
    layer.trainable = True 
model, Xception_all_arch2 = A.arch_TL_2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, base_model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_Xception_all_arch2'
outfile = open(filename,'wb')
pickle.dump(Xception_all_arch2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
flatten_6 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               9437696   
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                5130      
Total params: 30,304,306
Trainable params: 30,249,778
Non-trainable params: 54,528
_________________________________________________________________
Epoch 1/100
1595/1595 [==============================] - 193s 121ms/step - loss: 1.1806 - acc: 0.8241 - val_loss: 14.6117 - v

In [29]:
scores=[]

for folder in os.walk('/home/ralph/ml/Sages/Zaliczenie/final/results'):
    for file in folder[2]:
        filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/' + file
        infile = open(filename,'rb')
        a=pickle.load(infile)
        a.append(file)
        scores.append(a)
        infile.close()
              
# print(scores)
            
def getKey(item):
    return item[2]
scores = sorted(scores, key=getKey)    
        
for score in scores:
    print("Architektura: {} miałą acc na zbiorze testowym równe: {:.2f}".format(score[-1],score[2]))

Architektura: eval_arch_6 miałą acc na zbiorze testowym równe: 0.08
Architektura: eval_arch_8 miałą acc na zbiorze testowym równe: 0.09
Architektura: eval_arch_7 miałą acc na zbiorze testowym równe: 0.09
Architektura: eval_arch_9 miałą acc na zbiorze testowym równe: 0.10
Architektura: eval_arch_4 miałą acc na zbiorze testowym równe: 0.10
Architektura: eval_arch_3 miałą acc na zbiorze testowym równe: 0.10
Architektura: eval_arch_10 miałą acc na zbiorze testowym równe: 0.11
Architektura: eval_arch_5 miałą acc na zbiorze testowym równe: 0.11
Architektura: eval_arch_1 miałą acc na zbiorze testowym równe: 0.30
Architektura: eval_arch_2 miałą acc na zbiorze testowym równe: 0.39
Architektura: eval_VGG16_arch1 miałą acc na zbiorze testowym równe: 0.41
Architektura: eval_VGG16_arch2 miałą acc na zbiorze testowym równe: 0.42
Architektura: eval_Xception_arch3 miałą acc na zbiorze testowym równe: 0.43
Architektura: eval_VGG19_arch1 miałą acc na zbiorze testowym równe: 0.43
Architektura: eval_VGG19

Jak widzimy mimo trenowania całego modelu nie doszlismy nawet do tak dobrego wyniku, spróbuję teraz innego podejscia

### VGG16 na 224x224

Model będzie sie bardzo długo uczył, a z prac testowych wiem że 2 warstwowa sieć zadziałą tu najlepiej, dlatego uruchomię tylko architekturę 3

In [3]:
train_data_dir = '/home/ralph/ml/Sages/Zaliczenie/Wybrane/train/'
validation_data_dir = '/home/ralph/ml/Sages/Zaliczenie/Wybrane/validate/'
test_data_dir = '/home/ralph/ml/Sages/Zaliczenie/Wybrane/test/'

multiplier = 40 # 40 razy więcej obrazków wybierzemy z data generatora
batch_size = 32
w=224
h=224

train_datagen = ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                rotation_range=30,
                                horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(h, w),
                                                    batch_size=batch_size, 
                                                    class_mode='categorical',
                                                       shuffle=True)
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                        target_size=(h, w),
                                                        batch_size=220, # liczba wszystkich obrazków w zbiorze
                                                        class_mode='categorical',
                                                       shuffle=False)
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                        target_size=(h, w),
                                                        batch_size=203, # liczba wszystkich obrazków w zbiorze
                                                        class_mode='categorical',
                                                        shuffle=False)

num_classes = train_generator.num_classes
image_shape = train_generator.image_shape

nb_train_samples = train_generator.n
nb_validation_samples = validation_generator.n
nb_test_samples = test_generator.n

Found 1276 images belonging to 10 classes.
Found 220 images belonging to 10 classes.
Found 203 images belonging to 10 classes.


In [4]:
# VGG16 trzecia architektura
optimizer = Adam()

base_model = VGG16(weights='imagenet', include_top=False, input_shape=image_shape)

for layer in base_model.layers[1:]: 
    layer.trainable = False 
model, VGG16_224_arch3 = A.arch_TL_3(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, base_model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG16_224_arch3'
outfile = open(filename,'wb')
pickle.dump(VGG16_224_arch3,outfile)
outfile.close()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               12845568  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
___________________________

In [5]:
# Odblokowuje ostatani blok konwolucji
for layer in model.layers[0].layers[17:]: 
    layer.trainable = True
  
optimizer = Adam(0.00001, decay=0.00000001)

In [6]:
# VGG16 - druga faza
# Odblokowuje ostatani blok konwolucji
for layer in model.layers[0].layers[15:]: 
    layer.trainable = True
   
optimizer = Adam(0.00001, decay=0.00000001)

model, VGG16_224_arch3_2 = A.TL_phase2(image_shape, num_classes, callbacks, train_generator, nb_train_samples*multiplier,\
                       validation_generator,nb_validation_samples,batch_size,test_generator,nb_test_samples,\
                      regularizer,optimizer, model)
filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/eval_VGG16_224_arch3_2'
outfile = open(filename,'wb')
pickle.dump(VGG16_224_arch3_2,outfile)
outfile.close()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               12845568  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total para

In [7]:
scores=[]

for folder in os.walk('/home/ralph/ml/Sages/Zaliczenie/final/results'):
    for file in folder[2]:
        filename = '/home/ralph/ml/Sages/Zaliczenie/final/results/' + file
        infile = open(filename,'rb')
        a=pickle.load(infile)
        a.append(file)
        scores.append(a)
        infile.close()
              
# print(scores)
            
def getKey(item):
    return item[2]
scores = sorted(scores, key=getKey)    
        
for score in scores:
    print("Architektura: {} miałą acc na zbiorze testowym równe: {:.2f}".format(score[-1],score[2]))

Architektura: eval_arch_6 miałą acc na zbiorze testowym równe: 0.08
Architektura: eval_arch_8 miałą acc na zbiorze testowym równe: 0.09
Architektura: eval_arch_7 miałą acc na zbiorze testowym równe: 0.09
Architektura: eval_arch_9 miałą acc na zbiorze testowym równe: 0.10
Architektura: eval_arch_4 miałą acc na zbiorze testowym równe: 0.10
Architektura: eval_arch_3 miałą acc na zbiorze testowym równe: 0.10
Architektura: eval_arch_10 miałą acc na zbiorze testowym równe: 0.11
Architektura: eval_arch_5 miałą acc na zbiorze testowym równe: 0.11
Architektura: eval_arch_1 miałą acc na zbiorze testowym równe: 0.30
Architektura: eval_arch_2 miałą acc na zbiorze testowym równe: 0.39
Architektura: eval_VGG16_arch1 miałą acc na zbiorze testowym równe: 0.41
Architektura: eval_VGG16_arch2 miałą acc na zbiorze testowym równe: 0.42
Architektura: eval_Xception_arch3 miałą acc na zbiorze testowym równe: 0.43
Architektura: eval_VGG19_arch1 miałą acc na zbiorze testowym równe: 0.43
Architektura: eval_VGG19

### Koniec

Zatrzymam się w tym miejscu, mógłbym trenować bardziej złożone modele jak choćby wcześniej trenowany Xception ale mój sprzęt komputerowy nie pozwla na trenowanie obrazków w rozdzielczoscie 299x299 na tak duzych sieciach.

Zapiszę więc model za pomocą pickle i przygotuję prostą stronę we Flasku do oceny obrazków.

In [8]:
filename = '/home/ralph/ml/Sages/Zaliczenie/final/VGG16_224x224x3_D512_D128.model'
outfile = open(filename,'wb')
pickle.dump(model,outfile)
outfile.close()